# transformations

> Classes and functions that deal with transforming text during notebook export

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp markdown.transformations

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| hide
#| export
import io
import re
from typing import Callable, Iterable, Sequence

In [ ]:
# | export


class Transformer:
    """Base class for all content transformers."""

    def emit_before(self, stream: io.TextIOBase):
        """Implement this method on sub-classes to emit markdown \
            before the set of lines this transformer processes."""
        return

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        """Implement this method on sub-classes to modify the lines \
            this transformer processes."""
        return lines

    def emit_after(self, stream: io.TextIOBase):
        """Implement this method on sub-classes to emit markdown \
            after the set of lines this transformer processes."""
        return

In [ ]:
show_doc(Transformer.emit_before)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/markdown/transformations.py#L14){target="_blank" style="float:right; font-size:smaller"}

### Transformer.emit_before

>      Transformer.emit_before (stream:io.TextIOBase)

Implement this method on sub-classes to emit markdown             before the set of lines this transformer processes.

In [ ]:
show_doc(Transformer.process_lines)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/markdown/transformations.py#L19){target="_blank" style="float:right; font-size:smaller"}

### Transformer.process_lines

>      Transformer.process_lines (lines:Sequence[str])

Implement this method on sub-classes to modify the lines             this transformer processes.

In [ ]:
show_doc(Transformer.emit_after)

---

[source](https://github.com/spather/beetroot/blob/main/beetroot/markdown/transformations.py#L24){target="_blank" style="float:right; font-size:smaller"}

### Transformer.emit_after

>      Transformer.emit_after (stream:io.TextIOBase)

Implement this method on sub-classes to emit markdown             after the set of lines this transformer processes.

In [ ]:
# | export
class MultiTransformer:
    def __init__(self, transformers: Iterable[Transformer]):
        # Store the passed in transformers as a list 
        # so that we can later call reverse() on it.
        self.transformers = list(transformers)

    def emit_before(self, stream: io.TextIOBase):
        # Emit all the before output from the transformers.
        # Do it in reversed order so that the first transformer's
        # before output appears closest to the (transformed) lines
        for transformer in reversed(self.transformers):
            transformer.emit_before(stream)

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        # Pass the source lines through all the transformers
        for transformer in self.transformers:
            lines = transformer.process_lines(lines)
        return lines

    def emit_after(self, stream: io.TextIOBase):
        # Emit the after output from the transformers
        for transformer in self.transformers:
            transformer.emit_after(stream)

In [ ]:
# | export

def emit_with_transformation(
    transformer: Transformer,
    lines: Sequence[str],
    emit_lines_func: Callable[[Sequence[str], io.TextIOBase], None],
    stream: io.TextIOBase,
):
    transformer.emit_before(stream)

    emit_lines_func(transformer.process_lines(lines), stream)

    transformer.emit_after(stream)


In [ ]:
# Tests for emit_with_transformations()
class TransformerA(Transformer):
    def emit_before(self, stream: io.TextIOBase):
        stream.write('before A\n')

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        return [
            line.replace('a', 'A')
            for line in lines 
        ]

    def emit_after(self, stream: io.TextIOBase):
        stream.write('after A\n')

class TransformerB(Transformer):
    def emit_before(self, stream: io.TextIOBase):
        stream.write('before B\n')

    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        return [
            line.replace('b', 'B')
            for line in lines 
        ]

    def emit_after(self, stream: io.TextIOBase):
        stream.write('after B\n')

def dummy_emit_lines(lines: Sequence[str], stream: io.TextIOBase):
    for line in lines:
        stream.write(line)

lines = [
    'abCDefab\n',
    'ghiAbajkl\n'
    'mnoAAAAAABBBBBp\n'
]

stream = io.StringIO()

emit_with_transformation(
    transformer=MultiTransformer([TransformerA(), TransformerB()]),
    lines=lines,
    emit_lines_func=dummy_emit_lines,
    stream=stream
)

stream.seek(0)
output = stream.read()

expected = """\
before B
before A
ABCDefAB
ghiABAjkl
mnoAAAAAABBBBBp
after A
after B
"""

test_eq(output, expected)

In [ ]:
# | export
class ReplaceSingleDollarDelimiters(Transformer):
    """Transformer that replaces $ delimiters in inline latex\
        with \\\\( and \\\\)."""
    def process_lines(self, lines: Sequence[str]) -> Sequence[str]:
        regex = r"(?<!\$)\$(?!\$)(.*?[^\\])\$(?!\$)(?!\w)"
        replacement = r"\\\\(\1\\\\)"

        return [re.sub(regex, replacement, line) for line in lines]

In [ ]:
# Test ReplaceSingleDollarDelimiters

input = [
    'This is an expression in which the delimiters will be replaced: $a + b$\n',
    'These block delimiters should not be replaced: $$a + b$$\n'
]

output = ReplaceSingleDollarDelimiters().process_lines(input)

expected = [
    r'This is an expression in which the delimiters will be replaced: \\(a + b\\)''\n',
    'These block delimiters should not be replaced: $$a + b$$\n'
]

test_eq(output, expected)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()